In [5]:
import deep_image_matching as dim

params = {
    "dir": "./assets/example_cyprus",
    "pipeline": "sift+lightglue",
    "strategy": "matching_lowres",
    "quality": "medium",
    "tiling": "none",
    "camera_options": "./assets/example_cyprus/cameras.yaml",
    "openmvg": None,
}
config = dim.Config(params)

Run feature extraction and matching


In [6]:
matcher = dim.ImageMatcher(config)
feature_path, match_path = matcher.run()

Downloading: "https://github.com/cvg/LightGlue/releases/download/v0.1_arxiv/sift_lightglue.pth" to /home/francesco/.cache/torch/hub/checkpoints/sift_lightglue_v0-1_arxiv-pth
100%|██████████| 45.4M/45.4M [00:01<00:00, 44.7MB/s]


Loaded LightGlue model
2024-05-13 18:26:38 | [INFO    ] Running image matching with the following configuration:
2024-05-13 18:26:38 | [INFO    ]   Image folder: assets/example_cyprus/images
2024-05-13 18:26:38 | [INFO    ]   Output folder: assets/example_cyprus/results_sift+lightglue_matching_lowres_quality_medium
2024-05-13 18:26:38 | [INFO    ]   Number of images: 10
2024-05-13 18:26:38 | [INFO    ]   Matching strategy: matching_lowres
2024-05-13 18:26:38 | [INFO    ]   Image quality: MEDIUM
2024-05-13 18:26:38 | [INFO    ]   Tile selection: NONE
2024-05-13 18:26:38 | [INFO    ]   Feature extraction method: sift
2024-05-13 18:26:38 | [INFO    ]   Matching method: lightglue
2024-05-13 18:26:38 | [INFO    ]   Geometric verification: PYDEGENSAC
2024-05-13 18:26:38 | [INFO    ]   CUDA available: True
2024-05-13 18:26:38 | [INFO    ] Low resolution matching, generating pairs ..
Loaded SuperPoint model
2024-05-13 18:26:38 | [INFO    ] Extracting features from downsampled images...


100%|██████████| 10/10 [00:00<00:00, 16.52it/s]

2024-05-13 18:26:39 | [INFO    ] Matching downsampled images...



100%|██████████| 45/45 [00:00<00:00, 49.07it/s]

...

2024-05-13 18:26:43 | [WARNING ] Cannot find scores in assets/example_cyprus/results_sift+lightglue_matching_lowres_quality_medium/features.h5
2024-05-13 18:26:43 | [WARNING ] Cannot find scores in assets/example_cyprus/results_sift+lightglue_matching_lowres_quality_medium/features.h5
2024-05-13 18:26:43 | [WARNING ] Cannot find scores in assets/example_cyprus/results_sift+lightglue_matching_lowres_quality_medium/features.h5
2024-05-13 18:26:43 | [WARNING ] Cannot find scores in assets/example_cyprus/results_sift+lightglue_matching_lowres_quality_medium/features.h5


 93%|█████████▎| 26/28 [00:01<00:00, 12.14it/s]

2024-05-13 18:26:43 | [WARNING ] Cannot find scores in assets/example_cyprus/results_sift+lightglue_matching_lowres_quality_medium/features.h5
2024-05-13 18:26:43 | [WARNING ] Cannot find scores in assets/example_cyprus/results_sift+lightglue_matching_lowres_quality_medium/features.h5
2024-05-13 18:26:44 | [WARNING ] Cannot find scores in assets/example_cyprus/results_sift+lightglue_matching_lowres_quality_medium/features.h5
2024-05-13 18:26:44 | [WARNING ] Cannot find scores in assets/example_cyprus/results_sift+lightglue_matching_lowres_quality_medium/features.h5


100%|██████████| 28/28 [00:02<00:00, 13.08it/s]


2024-05-13 18:26:44 | [INFO    ] [Timer] | [matching] generate_pairs=24.904, extract_features=1.745, Match pair=0.087, Total execution=28.811
2024-05-13 18:26:44 | [INFO    ] [Timer] | [Deep Image Matching] Total execution=0.000


Export matches to COLMAP format


In [7]:
dim.io.export_to_colmap(
    img_dir=config.general["image_dir"],
    feature_path=feature_path,
    match_path=match_path,
    database_path=config.general["output_dir"] / "database.db",
    camera_config_path=config.general["camera_options"],
)

2024-05-13 18:26:44 | [WARNING ] Was not possible to load the first image to initialize cam0
2024-05-13 18:26:44 | [WARNING ] Was not possible to load the first image to initialize cam1


100%|██████████| 10/10 [00:00<00:00, 2216.98it/s]
28it [00:00, 12079.87it/s]            
25it [00:00, 10873.96it/s]            


Run reconstruction with pycolmap


In [8]:
model = dim.reconstruction.pycolmap_reconstruction(
    database_path=config.general["output_dir"] / "database.db",
    sfm_dir=config.general["output_dir"],
    image_dir=config.general["image_dir"],
    refine_intrinsics=False,
)

2024-05-13 18:26:44 | [INFO    ] Running 3D reconstruction...


I20240513 18:26:44.147128 271170 misc.cc:198] 
Loading database
I20240513 18:26:44.148154 271170 database_cache.cc:54] Loading cameras...
I20240513 18:26:44.148192 271170 database_cache.cc:64]  1 in 0.000s
I20240513 18:26:44.148211 271170 database_cache.cc:72] Loading matches...
I20240513 18:26:44.148278 271170 database_cache.cc:78]  25 in 0.000s
I20240513 18:26:44.148285 271170 database_cache.cc:94] Loading images...
I20240513 18:26:44.149093 271170 database_cache.cc:143]  10 in 0.001s (connected 8)
I20240513 18:26:44.149122 271170 database_cache.cc:154] Building correspondence graph...
I20240513 18:26:44.150086 271170 database_cache.cc:190]  in 0.001s (ignored 0)
I20240513 18:26:44.150133 271170 timer.cc:91] Elapsed time: 0.000 [minutes]
I20240513 18:26:44.150638 271170 misc.cc:198] 
Finding good initial image pair
I20240513 18:26:44.179556 271170 misc.cc:198] 
Initializing with image pair #7 and #8
I20240513 18:26:44.179932 271170 misc.cc:198] 
Global bundle adjustment
I20240513 18:

2024-05-13 18:26:44 | [INFO    ] Reconstructed 2 model(s).
2024-05-13 18:26:44 | [INFO    ] Largest model is #1 with 8 images.
2024-05-13 18:26:44 | [INFO    ] Exporting model #0...
2024-05-13 18:26:44 | [INFO    ] Exporting model #1...


I20240513 18:26:44.785068 271170 timer.cc:91] Elapsed time: 0.011 [minutes]
